In [1]:
import pandas as pd
import numpy as np
from thefuzz import process

In [41]:
movies = pd.read_csv('Data/movie_ratings.csv')
cast_df = pd.read_csv('/Users/parkerharalds/Downloads/title.crew.tsv', delimiter='\t')
name_df = pd.read_csv('/Users/parkerharalds/Downloads/name.basics.tsv', delimiter='\t')
rev_df = pd.read_csv('Data/rev_df.csv')

/var/folders/0k/y2qbjrj17tx6k7cdjb67jg100000gn/T/ipykernel_57177/1184675959.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('Data/movie_ratings.csv')


Creating a seperate row for each director:

In [42]:
#There are a lot of rows with multiple directors
rows_with_commas = cast_df[cast_df['directors'].str.contains(',')]

In [43]:
directors_df = cast_df

In [44]:
#breaking each row of director using ',' and creating a new row
directors_df['directors'] = directors_df['directors'].str.split(',')
directors_df = directors_df.explode('directors').reset_index(drop=True)

Merging all DFs:

In [45]:
movie_df = pd.merge(movies,directors_df,on='tconst')

In [50]:
main_df = pd.merge(movie_df,name_df,left_on='directors',right_on='nconst')

In [53]:
#creating name_id to join revenue and imdb data
main_df['name_id'] = main_df['primaryTitle'] + main_df['startYear'].astype(str)

In [54]:
#name_id create for revenue df merge
rev_df['date'] = pd.to_datetime(rev_df['date'])
rev_df['year'] = rev_df['date'].dt.year
rev_df['name_id'] = rev_df['primaryTitle'] + rev_df['year'].astype(str)

Using fuzzy merge to join data:

In [55]:
# Create a dictionary to store the best match from main_df for each name in num_df
match_dict = {}
for name in rev_df['name_id']:
   # Use extractOne to get the best match
    result = process.extractOne(name, main_df['name_id'])
    
    # Only proceed if result is valid
    if result:
        # Unpack result to get best_match and score, ignore any extras
        best_match, score = result[:2]
        if score > 90:  # Threshold for a good match
            match_dict[name] = best_match

# Map the matches from df2 to df1's 'name' column
rev_df['matched_name'] = rev_df['name_id'].map(match_dict)

In [56]:
merged_df = pd.merge(rev_df, main_df, left_on='matched_name', right_on='name_id', how='left')

In [65]:
drop_these = ['Rank','name_id_x','matched_name','tconst','titleType','primaryTitle_y','endYear'
              ,'writers','nconst','deathYear','name_id_y','knownForTitles','primaryProfession','directors']
df = merged_df.drop(columns=drop_these,index=1)

In [69]:
df.to_csv('directors.csv',index=False)